# FINAL REPORT ON THE COURSERA CAPSTONE PROJECT

## I- INTRODUCTION / BUSINESS PROBLEM

## II- DATA

This section presents the data I will use to solve the problem. I assume the city is the clermont-ferrand town.

### Definition of Foursquare credentials and version

In [2]:
CLIENT_ID = '1TCFP5PIX2MR0LG4BVYIJLL1PGV0Y3Y01IGK5DTHFIAFBF4F' # your Foursquare ID
CLIENT_SECRET = 'ASWAFPQNGFUO0C2G0PAQSKLJMRR5QMNTOHY3S0CFWE24SWEH' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1TCFP5PIX2MR0LG4BVYIJLL1PGV0Y3Y01IGK5DTHFIAFBF4F
CLIENT_SECRET:ASWAFPQNGFUO0C2G0PAQSKLJMRR5QMNTOHY3S0CFWE24SWEH


### Import necessary librairies

In [3]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.6.16          |           py36_0         148 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

### City (Clermont-Ferrand) search for latitude and longitude coordinates

In [4]:
address = ' Clermont-Ferrand, CF'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

45.7520738 2.9742462


### Explanation of data that will be used

### Definding an url 

In [6]:
#search_query = 'Italian'
radius = 500
#url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=1TCFP5PIX2MR0LG4BVYIJLL1PGV0Y3Y01IGK5DTHFIAFBF4F&client_secret=ASWAFPQNGFUO0C2G0PAQSKLJMRR5QMNTOHY3S0CFWE24SWEH&ll=45.7520738,2.9742462&v=20180604&radius=500&limit=30'

### Send the GET Request and examine the results

In [7]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d27aff8ad178900393d7618'},
 'response': {'venues': [{'id': '4c506162dcb5d13a365ce808',
    'name': 'Archipel Volcans',
    'location': {'address': '19 Route de Clermont',
     'lat': 45.74503779,
     'lng': 2.96901137,
     'labeledLatLngs': [{'label': 'display',
       'lat': 45.74503779,
       'lng': 2.96901137}],
     'distance': 882,
     'postalCode': '63122',
     'cc': 'FR',
     'city': 'Saint-Gènes-Champanelle',
     'state': 'Auvergne-Rhône-Alpes',
     'country': 'France',
     'formattedAddress': ['19 Route de Clermont',
      '63122 Saint-Gènes-Champanelle',
      'France']},
    'categories': [{'id': '4bf58dd8d48988d1fa931735',
      'name': 'Hotel',
      'pluralName': 'Hotels',
      'shortName': 'Hotel',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1562882040',
    'hasPerk': False},
   {'id': '4ccd60cdee23a143a37425a8',
  

### Get relevant part of JSON and transform it into a *pandas* dataframe

In [8]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4c506162dcb5d13a365ce808,19 Route de Clermont,FR,Saint-Gènes-Champanelle,France,NaN,882,"[19 Route de Clermont, 63122 Saint-Gènes-Champ...","[{'label': 'display', 'lat': 45.74503779, 'lng...",45.745038,2.969011,63122,Auvergne-Rhône-Alpes,Archipel Volcans,v-1562882040
1,[],False,4ccd60cdee23a143a37425a8,NaN,FR,NaN,France,NaN,1939,[France],"[{'label': 'display', 'lat': 45.76415255569107...",45.764153,2.956255,NaN,NaN,Col de Ceyssat,v-1562882040
2,"[{'id': '4bf58dd8d48988d159941735', 'name': 'T...",False,4fc38ba5e4b00dd50f29ffd2,NaN,FR,NaN,France,NaN,1956,[France],"[{'label': 'display', 'lat': 45.76486360505166...",45.764864,2.956972,NaN,NaN,Chemin Des Muletiers,v-1562882040
3,"[{'id': '4bf58dd8d48988d1ee931735', 'name': 'H...",False,539b34d7498ea14ee54a581a,NaN,FR,NaN,France,NaN,2356,[France],"[{'label': 'display', 'lat': 45.74145, 'lng': ...",45.741450,2.948009,NaN,NaN,Gite La Cîme des Bois,v-1562882040
4,"[{'id': '4bf58dd8d48988d1e6941735', 'name': 'G...",False,4feff191e4b0e198331310b5,NaN,FR,NaN,France,NaN,2855,[France],"[{'label': 'display', 'lat': 45.753599842237, ...",45.753600,3.010938,NaN,NaN,GOLF de ROYAT-CHARADE,v-1562882040


### Define information of interest and filter dataframe

In [9]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Archipel Volcans,Hotel,19 Route de Clermont,FR,Saint-Gènes-Champanelle,France,NaN,882,"[19 Route de Clermont, 63122 Saint-Gènes-Champ...","[{'label': 'display', 'lat': 45.74503779, 'lng...",45.745038,2.969011,63122,Auvergne-Rhône-Alpes,4c506162dcb5d13a365ce808
1,Col de Ceyssat,None,NaN,FR,NaN,France,NaN,1939,[France],"[{'label': 'display', 'lat': 45.76415255569107...",45.764153,2.956255,NaN,NaN,4ccd60cdee23a143a37425a8
2,Chemin Des Muletiers,Trail,NaN,FR,NaN,France,NaN,1956,[France],"[{'label': 'display', 'lat': 45.76486360505166...",45.764864,2.956972,NaN,NaN,4fc38ba5e4b00dd50f29ffd2
3,Gite La Cîme des Bois,Hostel,NaN,FR,NaN,France,NaN,2356,[France],"[{'label': 'display', 'lat': 45.74145, 'lng': ...",45.741450,2.948009,NaN,NaN,539b34d7498ea14ee54a581a
4,GOLF de ROYAT-CHARADE,Golf Course,NaN,FR,NaN,France,NaN,2855,[France],"[{'label': 'display', 'lat': 45.753599842237, ...",45.753600,3.010938,NaN,NaN,4feff191e4b0e198331310b5
5,Bibliothèque L'écume des jours,None,NaN,FR,Orcines,France,NaN,4429,"[63870 Orcines, France]","[{'label': 'display', 'lat': 45.782302133739, ...",45.782302,3.011337,63870,Auvergne,51b86e47498e380f209b39aa
6,Chapelle St aubin,Hot Spring,NaN,FR,NaN,France,NaN,1552,[France],"[{'label': 'display', 'lat': 45.74755262773890...",45.747553,2.993154,NaN,NaN,4f9faa1ee4b05255286e3010
7,Logis Le Montania Hotel Nebouzat,None,Les Quatre Routes,FR,Nébouzat,France,NaN,6771,"[Les Quatre Routes, Nebouzat, France]","[{'label': 'display', 'lat': 45.715968, 'lng':...",45.715968,2.904106,NaN,Auvergne,4bd8add32ecdce7231e9d0f2
8,Les délices d'orcines,Asian Restaurant,22 chemin de Bellevue,FR,Orcines,France,NaN,5484,"[22 chemin de Bellevue, 63870 Orcines, France]","[{'label': 'display', 'lat': 45.7814, 'lng': 3...",45.781400,3.031000,63870,Auvergne,50658788e4b042e2ccf566df
9,Complexe Sportif,Basketball Court,60 route de Limoges,FR,Orcines,France,NaN,4883,"[60 route de Limoges, 63870 Orcines, France]","[{'label': 'display', 'lat': 45.7880005681259,...",45.788001,3.010345,63870,Auvergne,550ebe69498ed654c6019138


### Discussion